In [103]:
# common python modules
import sys
import math
#import warnings
#warnings.filterwarnings("ignore")


# EDA and data visualization modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [104]:
data = pd.read_csv('data_files/fifa21_male2.csv')
data.columns = [i.replace(' ', '_').lower() for i in data.columns]

/Users/sebastianvier/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [183]:
# Reusable functions

def slice_by_values(columns, init, end):
    init = list(columns).index(init)
    end = list(columns).index(end)
    return list(columns[init:end + 1])

def remove_sign(val, sign):
        if (type(val) == str) and (val[0] == sign):
            return val[1:]
        else:
            return val

def euros2numbers(val):
    """Turn values with € sign (€20K) into float values for math calculations"""
     
    def convert2number(val):
        if type(val) == str:
            if val[-1] == 'K':
                return float(val[:-1]) * (10 ** 3)
            if val[-1] == 'M':
                return float(val[:-1]) * (10 ** 6)
            else:
                return val
        else:
            return val
    
    val = remove_sign(val, '€')
    val = convert2number(val)
    return val

def devide_position_generator(df, both=False):
    def devide_position_columns(value, pos):
        array =  value.split('+')
        return int(array[pos].strip())
    
    for column in df.columns:
        yield df[column].apply(lambda x : devide_position_columns(x,0))
        if both:
            yield df[column].apply(lambda x : devide_position_columns(x,1))
            
def change_first_val(arr, f_value):
    lst = [f_value]
    for value in arr:
        if value != f_value:
            lst.append(value)
    return lst

def add_monetary_2_numeric(categorical_df):
    for column in categorical_df.columns:
        col_first_symbol = categorical_df[column].iloc[0][0] #of each row of each column
        if col_first_symbol == '€':
            yield categorical[column].apply(lambda x : euros2numbers(x))

def add_position_2_numeric(categorical_df):
    columns = slice_by_values(categorical_df.columns,init='ls',end='gk')
    
    transformed_df = pd.concat([i for i in devide_position_generator(categorical_df[columns])], axis=1)
    result = pd.concat([numerical, transformed_df], axis=1)
    return result, columns



def filter_condition_3(value, threshold=.10):
    if value[-1] != 'K':
        return False
    if int(value[1:-1]) > (escudero_wage + escudero_wage*threshold):
        return False
    if int(value[1:-1]) < (escudero_wage - escudero_wage*threshold):
        return False
    else:
        return True
    


### Some information about the data

In [106]:
# How many teams have 20 or more players?
print('Number of teams with more than 20 players:')
num_teams_20 = len(data.iloc[:,5].value_counts()[data.iloc[:,5].value_counts() >= 20])
print(num_teams_20)

# Number of total teams
print('\nTotal number of teams')
num_teams = data.iloc[:,5].nunique()
print(num_teams)

Number of teams with more than 20 players:
446

Total number of teams
917


In [107]:
# Percentage of teams with less 20 players or more
print('\nPercentage of teams with 20 players or more')
print(round(num_teams_20/num_teams,2))


Percentage of teams with 20 players or more
0.49


##### This means that more than half of the teams in the data set have less than 20 players. Probably there is some players missing

### Decide the best replacement for a transfered player in Sevilla FC

 First we are going to decide what player is going to be transfered.
 We select a player who is leaving on 2021 and has the highest wage.
A player with a high wage is likely a player who is playing most matches.

##### But first we need to convert the column wage.

In [108]:
sevilla_fc = data[data['club'] == 'Sevilla FC']
sorted_index = sevilla_fc['wage'].map(lambda x : int(x[1:-1])).sort_values(ascending=False).index
## This codes strip the K and € sing gets the index to sort the players
sevilla_fc.loc[sorted_index[:10]][['name', 'nationality', 'contract', 'wage', 'age']]
## That index is used then on the sevilla_fc to sort the playes by wage.
## The [:10] is used to only get the first 10 players.

,name,nationality,contract,wage,age
569,Jesús Navas,Spain,2017 ~ 2021,€43K,34
7631,M. Acuña,Argentina,2020 ~ 2024,€41K,28
4593,L. Ocampos,Argentina,2019 ~ 2024,€40K,25
6702,Diego Carlos,Brazil,2019 ~ 2024,€39K,27
4048,Suso,Spain,2020 ~ 2025,€39K,26
1061,I. Rakitić,Croatia,2020 ~ 2024,€39K,32
1925,Fernando,Brazil,2019 ~ 2022,€39K,32
2482,L. de Jong,Netherlands,2019 ~ 2023,€38K,29
2845,Escudero,Spain,2015 ~ 2021,€31K,30
7624,Joan Jordán,Spain,2019 ~ 2023,€30K,25


So the player the players who is leaving is **Sergio Escudero**. Actually he is not playing in Sevilla anymore as to 2021.

### Candidates

 ###### First we check the players that have the same best position and the best position of Escudero

In [109]:
bp_escudero = data.iloc[2845]['bp']
position_escudero = data.iloc[2845]['position']
escudero_wage = int(data.iloc[2845]['wage'][1:-1])

## Amount of players that have exactly the same position

## This is to reduce the data I am currently using.
info_data = data[slice_by_values(data.columns, 'id', 'contract')]

# Same position and same best position
condition_1 = info_data['position'] == position_escudero
condition_2 = info_data['bp'] == bp_escudero



###### Now we check at a player with a similar Wage and whos age is no more than 33

In [110]:
# Around the same wage condition_3
    
condition_3 = info_data['wage'].apply(lambda x : filter_condition_3(x, .40)) 
condition_4 = info_data['age'] < 33

## Create a global condition
conditions = condition_1 & condition_2 & condition_3 & condition_4
candidates = data[conditions]

In [111]:
candidates.shape

(14, 107)

### Cleaning the data

In [522]:
## Delete all unnecesary columns
unnesary_columns = ['player_photo', 'club_logo', 'flag_photo', 'team_&_contract', 'loan_date_end']
numerical = candidates.select_dtypes(np.number).drop('id', axis=1)
categorical = candidates.select_dtypes(exclude=np.number).drop(unnesary_columns,axis=1, errors='ignore')

In [523]:
## Transform position in the court to the numeric values and drop them from categorical
numerical, droping_columns = add_position_2_numeric(categorical)
categorical = categorical.drop(droping_columns, axis=1,errors='ignore')

In [524]:
## Change the values concerning money and add them to numerical
monetary = pd.concat([i for i in add_monetary_2_numeric(categorical)], axis=1)
numerical = pd.concat([numerical,monetary],axis=1)

#### Eliminating collinearity in the data

In [525]:
print(numerical.shape)

(14, 84)


In [534]:
def add_values_2_drop(df, threshold=.1):
    for i in range(1, len(df)):
        if df[i] > (1-threshold):
#             print(f'Correlation found between {df.index[0]} and {df.index[i]}')
            drop_list.append(df.index[i])

def drop_columns_from_init():
    for column in initial_columns:
        if column in drop_list:
            initial_columns.remove(column)
            
drop_list = []
checked_columns = []
initial_columns = list(numerical.columns)
corr = numerical[initial_columns].corr().iloc[0,:] 


while len(initial_columns) > 2:
    add_values_2_drop(corr, threshold=.03)
    checked_columns.append(initial_columns[0])
    initial_columns = initial_columns[1:]
    drop_columns_from_init()
    corr = numerical[initial_columns].corr().iloc[0,:]
    
def eliminate_collinearity(data):
    pass

In [537]:
numerical.shape

(14, 59)